In [14]:
import matplotlib.pyplot as plt
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
#import soundfile as sf
import os
import glob
import numpy as np
from PIL import Image
from scipy.fftpack import fft
from scipy.misc import imresize
from IPython.display import Image
from IPython.core.display import HTML
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
import operator
from PIL import Image
from pydub import AudioSegment
from pydub.utils import make_chunks
import seaborn as sns
import cv2
import random
import tensorflow_hub as hub
import time
%matplotlib inline
# tf.logging.set_verbosity(tf.logging.INFO)
tf.logging.set_verbosity(tf.logging.ERROR)

/home/rainbowww5/anaconda3/lib/python3.5/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [13]:
audio_path = './input/train/audio/'
pict_Path = './input/picts/train/'
test_pict_Path = './input/picts/test/'
test_audio_path = './input/test/audio/'
samples = []

# os.system('rm -rf ./input/picts/test')
# os.system('rm -rf ./input/picts/train')
# os.mkdir('./input/picts/test')
# os.mkdir('./input/picts/train')

In [5]:
subFolderList = []
for x in os.listdir(audio_path):
    if os.path.isdir(audio_path + '/' + x):
        subFolderList.append(x)

In [ ]:
# def split_wav(wav_path, saved_location)
#     myaudio = AudioSegment.from_file(wav_path, "wav")
#     chunk_length_ms = 1000
#     chunks = make_chunks(myaudio, chunk_length_ms)
#     for i, chunk in enumerate(chunks):
#         chunk_name = wav_path[-4:]
#         chunk.export(chunk_name, format="wav")
        
# wav_path = "./input/train/audio/_background_noise_"

In [15]:
def flip_image(image_path, saved_location):
    image_obj = Image.open(image_path)
    rotated_image = image_obj.transpose(Image.FLIP_LEFT_RIGHT)
    rotated_image.save(saved_location)

    
def rotate_image(image_path, degrees, saved_location):
    image_obj = Image.open(image_path)
    rotated_image = image_obj.rotate(degrees)
    rotated_image.save(saved_location)

silence = "./input/picts/train/silence/"
for x in os.listdir(silence):
    if '.jpg' in x:
        flip_image(silence + x, silence + 'flip_' + x)
        rotate_image(silence + x, 90, silence + 'rotate_90' + x)
        rotate_image(silence + x, 180, silence + 'rotate_180' + x)
        rotate_image(silence + x, 270, silence + 'rotate_270' + x)

In [4]:
if not os.path.exists(pict_Path):
    os.makedirs(pict_Path)

if not os.path.exists(test_pict_Path):
    os.makedirs(test_pict_Path)


labels_to_keep = ['yes', 'no', 'up', 'down', 'left',
                  'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']

# for f in labels_to_keep:
#     os.mkdir(pict_Path + f)
    
# for f in labels_to_keep:
#     os.mkdir(test_pict_Path + f)

In [5]:
sample_audio = []
total = 0
for x in subFolderList:
    
    # get all the wave files
    all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
    total += len(all_files)
    # collect the first file from each dir
    sample_audio.append(audio_path  + x + '/'+ all_files[0])
    
    # show file counts
    print('count: %d : %s' % (len(all_files), x ))
print(total)

count: 1733 : tree
count: 2357 : five
count: 1746 : dog
count: 1742 : happy
count: 2372 : go
count: 2367 : right
count: 395 : _background_noise_
count: 1746 : marvin
count: 2375 : no
count: 2352 : eight
count: 2370 : one
count: 1713 : bed
count: 1731 : bird
count: 2364 : nine
count: 1745 : wow
count: 2373 : two
count: 2367 : on
count: 2380 : stop
count: 1750 : house
count: 2353 : left
count: 2377 : seven
count: 2356 : three
count: 1734 : sheila
count: 2376 : zero
count: 1733 : cat
count: 2372 : four
count: 2357 : off
count: 2377 : yes
count: 2359 : down
count: 2375 : up
count: 2369 : six
65116


In [6]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

In [7]:
def wav2img(wav_path, targetdir='', figsize=(4,4)):
    """
    takes in wave file path
    and the fig size. Default 4,4 will make images 288 x 288
    """
#     fig = plt.figure(figsize=figsize)
    # use soundfile library to read in the wave files
    samplerate, test_sound  = wavfile.read(wav_path)
    _, spectrogram = log_specgram(test_sound, samplerate)
    
    space = spectrogram.shape[0]
    if space < 161:
        spectrogram = np.pad(spectrogram, ((161-space,0), (0,0)), mode='constant', constant_values=0)
    spectrogram = imresize(spectrogram, (96,96))
#     spectrogram = tf.reshape(spectrogram, [-1, 161, 161, 1])
#     spectrogram = tf.image.resize_images(spectrogram, (96, 96))
#     spectrogram = tf.image.grayscale_to_rgb(spectrogram)
    
    ## create output path
    output_file = wav_path.split('/')[-1].split('.wav')[0]
    output_file = targetdir +'/'+ output_file
    #plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.imsave('%s.jpg' % output_file, spectrogram)
    plt.close()
    return spectrogram

In [8]:
def wav2img_waveform(wav_path, targetdir='', figsize=(4,4)):
    samplerate,test_sound  = wavfile.read(sample_audio[0])
    fig = plt.figure(figsize=figsize)
    plt.plot(test_sound)
    plt.axis('off')
    output_file = wav_path.split('/')[-1].split('.wav')[0]
    output_file = targetdir +'/'+ output_file
    plt.savefig('%s.png' % output_file)
    plt.close()

In [9]:
labels_to_keep = ['yes', 'no', 'up', 'down', 'left',
                  'right', 'on', 'off', 'stop', 'go']

test_labels = ['yes', 'no', 'up', 'down', 'left',
                  'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']

label_dict = {'yes':0, 'no':1, 'up':2, 'down':3, 'left':4,
                  'right':5, 'on':6, 'off':7, 'stop':8, 'go':9, 'silence':10, 'unknown':11}

In [10]:
# for i, x in enumerate(['yes', 'no']):
#         all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
#         for file in all_files[:]:
#             wav2img(audio_path + x + '/' + file, './input/yes_no/' + x)

In [11]:
# for i, x in enumerate(subFolderList):
#     if x in labels_to_keep:
#         print(i, ':', x)
#         # get all the wave files
#         all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
#         for file in all_files[:]:
#             wav2img(audio_path + x + '/' + file, pict_Path + x)
#     elif x == '_background_noise_':
#         print(i, ':', x)
#         # get all the wave files
#         all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
#         for file in all_files[:]:
#             wav2img(audio_path + x + '/' + file, "./input/picts/train/silence")
#     else:
#         print(i, ':', x)
#         all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
#         for file in all_files[:]:
#             wav2img(audio_path + x + '/' + file, "./input/picts/train/unknown")

In [12]:
# for i, x in enumerate(subFolderList):
#     if x in labels_to_keep:
#         print(i, ':', x)
#         # get all the wave files
#         all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
#         for file in all_files[101:120]:
#             wav2img(audio_path + x + '/' + file, test_pict_Path + x)
#     else:
#         all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
#         for file in all_files[16:20]:
#             wav2img(audio_path + x + '/' + file, "./input/picts/test/unknown")

In [13]:
# test = tf.image.decode_png("./input/picts/train/down/00b01445_nohash_1.png",3)
# sess = tf.Session()
# print(sess.run(test))

In [14]:
train_images = []
train_labels = []
for i, x in enumerate(test_labels): #choose the classes to train
    all_files = [y for y in os.listdir(pict_Path + x) if '.jpg' in y]
    for file in all_files[:1000]:
        train_images.append(cv2.imread(pict_Path + x + '/' + file))
        train_labels.append(label_dict[x])

train_images = np.array(train_images, dtype="float32")
train_labels = np.array(train_labels)
print(train_images.shape)
print(type(train_labels))

(11395, 96, 96, 3)
<class 'numpy.ndarray'>


In [15]:
print(train_labels.shape)

(11395,)


In [16]:
train_labels.reshape((1, -1))
train_labels = np.array(train_labels, dtype="int32")
randomize = np.arange(len(train_labels))
np.random.shuffle(randomize)
print(train_labels)
labels = train_labels[randomize]
features = train_images[randomize]
print(labels)

[ 0  0  0 ..., 11 11 11]
[ 6  5 10 ...,  8  2  3]


In [17]:
print(len(labels))

11395


In [18]:
x = len(labels)
x *= 0.8
x = int(x)
training_examples = features[:x]
validation_examples = features[x:]

training_targets = labels[:x]
validation_targets = labels[x:]
print(training_targets)
print(validation_targets)

[ 6  5 10 ...,  7  4  5]
[2 7 9 ..., 8 2 3]
